In [33]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd


In [34]:
df=pd.read_csv('coin_positions.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2198758 entries, 0 to 2198757
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   Unnamed: 0  int64 
 1   Coin 0      object
 2   Coin 1      object
 3   Coin 2      object
dtypes: int64(1), object(3)
memory usage: 67.1+ MB


In [35]:
df=df.drop(columns=[df.columns[0]])
df.head()

,Coin 0,Coin 1,Coin 2
0,80.25--63.23712636181692,89.25--29.75,89.25--98.31284390112754
1,79.5--63.47425272363384,88.5--30.5,88.5--97.62568780225507
2,78.75--63.71137908545076,87.75--31.25,87.75--96.93853170338261
3,78.0--63.94850544726768,87.0--32.0,87.0--96.25137560451014
4,77.25--64.1856318090846,86.25--32.75,86.25--95.56421950563768


In [36]:
for coin in df.columns:
    df[[f"{coin}_x", f"{coin}_y"]] = df[coin].str.split("--", expand=True).astype(float)
    # df[f"{coin}_x"] = df[f"{coin}_x"].round(8)
    # df[f"{coin}_y"] = df[f"{coin}_y"].round(8)

In [37]:
df.head()

,Coin 0,Coin 1,Coin 2,Coin 0_x,Coin 0_y,Coin 1_x,Coin 1_y,Coin 2_x,Coin 2_y
0,80.25--63.23712636181692,89.25--29.75,89.25--98.31284390112754,80.25,63.237126,89.25,29.75,89.25,98.312844
1,79.5--63.47425272363384,88.5--30.5,88.5--97.62568780225507,79.50,63.474253,88.50,30.50,88.50,97.625688
2,78.75--63.71137908545076,87.75--31.25,87.75--96.93853170338261,78.75,63.711379,87.75,31.25,87.75,96.938532
3,78.0--63.94850544726768,87.0--32.0,87.0--96.25137560451014,78.00,63.948505,87.00,32.00,87.00,96.251376
4,77.25--64.1856318090846,86.25--32.75,86.25--95.56421950563768,77.25,64.185632,86.25,32.75,86.25,95.564220


In [38]:
df.head()

,Coin 0,Coin 1,Coin 2,Coin 0_x,Coin 0_y,Coin 1_x,Coin 1_y,Coin 2_x,Coin 2_y
0,80.25--63.23712636181692,89.25--29.75,89.25--98.31284390112754,80.25,63.237126,89.25,29.75,89.25,98.312844
1,79.5--63.47425272363384,88.5--30.5,88.5--97.62568780225507,79.50,63.474253,88.50,30.50,88.50,97.625688
2,78.75--63.71137908545076,87.75--31.25,87.75--96.93853170338261,78.75,63.711379,87.75,31.25,87.75,96.938532
3,78.0--63.94850544726768,87.0--32.0,87.0--96.25137560451014,78.00,63.948505,87.00,32.00,87.00,96.251376
4,77.25--64.1856318090846,86.25--32.75,86.25--95.56421950563768,77.25,64.185632,86.25,32.75,86.25,95.564220


In [39]:
df=df.drop(columns=df.columns[:3])

In [40]:
df.head()

,Coin 0_x,Coin 0_y,Coin 1_x,Coin 1_y,Coin 2_x,Coin 2_y
0,80.25,63.237126,89.25,29.75,89.25,98.312844
1,79.50,63.474253,88.50,30.50,88.50,97.625688
2,78.75,63.711379,87.75,31.25,87.75,96.938532
3,78.00,63.948505,87.00,32.00,87.00,96.251376
4,77.25,64.185632,86.25,32.75,86.25,95.564220


In [41]:
data = torch.tensor(df.values, dtype=torch.float32)

sequence_length = 5
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        sequence = data[i:i + seq_length]
        sequences.append(sequence)
    return torch.stack(sequences)

sequences = create_sequences(data, sequence_length)


In [42]:
dataset = TensorDataset(sequences)
train_loader = DataLoader(dataset, batch_size=32, shuffle=False)

In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class Encoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, latent_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return x

class Decoder(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim, 64)
        self.fc2 = nn.Linear(64, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class AutoencoderLSTM(nn.Module):
    def __init__(self, input_dim, latent_dim, sequence_length):
        super(AutoencoderLSTM, self).__init__()
        self.encoder = Encoder(input_dim, latent_dim)
        self.lstm = nn.LSTM(latent_dim, latent_dim, batch_first=True)
        self.decoder = Decoder(latent_dim, input_dim)

    def forward(self, x):
        batch_size, seq_len, input_dim = x.size()
        x = x.view(batch_size * seq_len, input_dim)  # Flatten for encoding
        encoded = self.encoder(x)  # Encode each frame
        encoded = encoded.view(batch_size, seq_len, -1)  # Reshape for LSTM
        lstm_out, _ = self.lstm(encoded)  # Process with LSTM
        decoded = self.decoder(lstm_out[:, -1, :])  # Decode only last step
        return decoded

In [45]:
# Set parameters
input_dim = 6  # Number of x, y positions (per coin)
latent_dim = 64
sequence_length = 5
learning_rate = 0.001
num_epochs = 30

device = torch.device("cuda")
model = AutoencoderLSTM(input_dim=input_dim, latent_dim=latent_dim, sequence_length=sequence_length).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [46]:
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for sequences in train_loader:
        sequences = sequences[0].to(device)  # Extract data from DataLoader
        optimizer.zero_grad()
        output = model(sequences)
        target = sequences[:, -1, :]  # Actual next frame
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(train_loader)}")

Epoch [1/30], Loss: 746.2786789552159
Epoch [5/30], Loss: 59.79487263829874
Epoch [6/30], Loss: 45.13632757863207
Epoch [7/30], Loss: 33.91725440772218
Epoch [8/30], Loss: 26.58611898006927
Epoch [9/30], Loss: 22.041707214790062
Epoch [10/30], Loss: 20.50082183377365
Epoch [11/30], Loss: 18.899644665290914
Epoch [12/30], Loss: 17.564322362881942
Epoch [13/30], Loss: 16.403425108811625
Epoch [14/30], Loss: 16.32355385845831
Epoch [15/30], Loss: 15.2603948959652
Epoch [16/30], Loss: 15.185990816957514
Epoch [17/30], Loss: 13.706486382490514
Epoch [18/30], Loss: 13.292922259273917
Epoch [19/30], Loss: 12.925332653884947
Epoch [20/30], Loss: 13.41690026911189
Epoch [21/30], Loss: 12.272624859662965
Epoch [22/30], Loss: 13.015739100938658
Epoch [23/30], Loss: 12.031150908230202
Epoch [24/30], Loss: 11.620155580550168
Epoch [25/30], Loss: 11.66984432343985
Epoch [26/30], Loss: 11.563688932168313
Epoch [27/30], Loss: 11.463320853957146
Epoch [28/30], Loss: 11.390864067202173
Epoch [29/30], Lo

In [ ]:
# Prediction example
model.eval()
with torch.no_grad():
    for sequences in train_loader:
        sequences = sequences[0].to(device)
        output = model(sequences)
        print("Predicted next frame:", output.cpu().numpy())
        break  # Remove this if you want to loop 

In [21]:
model.load_state_dict(torch.load('model_weights.pth'))

<All keys matched successfully>

In [22]:
import torch
import pandas as pd

# Assuming your model and DataLoader (train_loader) are already set up
device = torch.device("cuda")
model = model.to(device)

# Parameters
num_predictions = 500000  # Number of frames to predict

# Get the initial sequence from the train_loader (first batch)
initial_sequence = None
for batch in train_loader:
    initial_sequence = batch[0]  # Assuming batch[0] contains the frames
    break  # Use only the first batch

initial_sequence = initial_sequence.to(device)  # Shape: (batch_size, sequence_length, num_coins * 2)
input_sequence = initial_sequence[0].unsqueeze(0)  # Shape: (1, sequence_length, num_coins * 2)

# Determine the number of coins based on input shape
sequence_length, num_features = input_sequence.shape[1], input_sequence.shape[2]
num_coins = num_features // 2

# Initialize list to store predictions
predicted_frames = []

# Set the model to evaluation mode
model.eval()

with torch.no_grad():
    for _ in range(num_predictions):
        # Predict the next frame
        predicted_frame = model(input_sequence)  # Shape: (1, num_coins * 2)
        
        # Format the predicted frame as "x--y" for each coin
        formatted_frame = []
        for i in range(num_coins):
            x = predicted_frame[0, i * 2].item()
            y = predicted_frame[0, i * 2 + 1].item()
            formatted_frame.append(f"{x:.8f}--{y:.8f}")
        
        # Append the formatted frame to the list for saving later
        predicted_frames.append(formatted_frame)
        
        # Update the input sequence by removing the oldest frame and appending the new prediction
        input_sequence = torch.cat([input_sequence[:, 1:], predicted_frame.unsqueeze(1)], dim=1)

# Convert predicted frames to DataFrame
column_names = [f"Coin {i}" for i in range(num_coins)]
predicted_frames_df = pd.DataFrame(predicted_frames, columns=column_names)

# Save to CSV
predicted_frames_df.to_csv("coin_positions2.csv", index=True)
print("Predicted frames saved to predicted_carrom_frames.csv")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (5x5 and 8x64)

In [46]:
torch.save(model.state_dict(), 'model_weights.pth')